In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import os
import pandas as pd
from pyspark.sql.types import IntegerType, LongType, DoubleType, BooleanType, DateType

In [2]:
# Probably need to change this if your on MAC
spark = (
    SparkSession.builder.appName("Preprocessing")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '4g')
    .config('spark.executor.memory', '2g')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
24/09/04 23:44:26 WARN Utils: Your hostname, Andy-PC resolves to a loopback address: 127.0.1.1; using 172.24.73.29 instead (on interface eth0)
24/09/04 23:44:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/04 23:44:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
transactions1 = spark.read.parquet("../data/tables/project-2-bnpl-tables-part2/tables/transactions_20210228_20210827_snapshot")
transactions2 = spark.read.parquet("../data/tables/project-2-bnpl-tables-part3/tables/transactions_20210828_20220227_snapshot")
transactions3 = spark.read.parquet("../data/tables/project-2-bnpl-tables-part4/tables/transactions_20220228_20220828_snapshot")
transactions = transactions1.union(transactions2).union(transactions3)
transactions.show()

+-------+------------+------------------+--------------------+--------------+
|user_id|merchant_abn|      dollar_value|            order_id|order_datetime|
+-------+------------+------------------+--------------------+--------------+
|  18478| 62191208634|63.255848959735246|949a63c8-29f7-4ab...|    2021-08-20|
|      2| 15549624934| 130.3505283105634|6a84c3cf-612a-457...|    2021-08-20|
|  18479| 64403598239|120.15860593212783|b10dcc33-e53f-425...|    2021-08-20|
|      3| 60956456424| 136.6785200286976|0f09c5a5-784e-447...|    2021-08-20|
|  18479| 94493496784| 72.96316578355305|f6c78c1a-4600-4c5...|    2021-08-20|
|      3| 76819856970|  448.529684285612|5ace6a24-cdf0-4aa...|    2021-08-20|
|  18479| 67609108741|  86.4040605836911|d0e180f0-cb06-42a...|    2021-08-20|
|      3| 34096466752| 301.5793450525113|6fb1ff48-24bb-4f9...|    2021-08-20|
|  18482| 70501974849| 68.75486276223054|8505fb33-b69a-412...|    2021-08-20|
|      4| 49891706470| 48.89796461900801|ed11e477-b09f-4ae...|  

In [4]:
tbl_consumer = spark.read.option("header", True).option("delimiter", "|").csv("../data/tables/project-2-bnpl-tables-part1/tables/tbl_consumer.csv")
print("Number of counts before preprocessing: " + str(tbl_consumer.count()))

tbl_consumer = tbl_consumer.drop("address") # dropped address column
tbl_consumer = tbl_consumer.filter(F.col("state").isin(["NSW", "WA", "NT", "VIC", "TAS", "QLD", "SA",   "ACT"]))   
tbl_consumer = tbl_consumer.filter(F.col("gender").isin(["Female", "Male", "Undisclosed"]))
tbl_consumer = tbl_consumer.withColumn("postcode", tbl_consumer["postcode"].cast(IntegerType()))

# Looked up all the valid postcodes in Australia. But some of them are considered LVR or PO boxes only, but I kept them in anyways idk.
tbl_consumer = tbl_consumer.filter(
    ((F.col("postcode") >= 1000) & (F.col("postcode") <= 1999)) |  
    ((F.col("postcode") >= 2000) & (F.col("postcode") <= 2599)) |
    ((F.col("postcode") >= 2619) & (F.col("postcode") <= 2899)) |
    ((F.col("postcode") >= 2921) & (F.col("postcode") <= 2999)) |
    ((F.col("postcode") >= 200) & (F.col("postcode") <= 299)) |  
    ((F.col("postcode") >= 2600) & (F.col("postcode") <= 2618)) |
    ((F.col("postcode") >= 2900) & (F.col("postcode") <= 2920)) |
    ((F.col("postcode") >= 3000) & (F.col("postcode") <= 3999)) |
    ((F.col("postcode") >= 8000) & (F.col("postcode") <= 8999)) |  
    ((F.col("postcode") >= 4000) & (F.col("postcode") <= 4999)) |
    ((F.col("postcode") >= 9000) & (F.col("postcode") <= 9999)) |  
    ((F.col("postcode") >= 5000) & (F.col("postcode") <= 5799)) |
    ((F.col("postcode") >= 5800) & (F.col("postcode") <= 5999)) |  
    ((F.col("postcode") >= 6000) & (F.col("postcode") <= 6797)) |
    ((F.col("postcode") >= 6800) & (F.col("postcode") <= 6999)) |  
    ((F.col("postcode") >= 7000) & (F.col("postcode") <= 7799)) |
    ((F.col("postcode") >= 7800) & (F.col("postcode") <= 7999)) |  
    ((F.col("postcode") >= 800) & (F.col("postcode") <= 899)) |
    ((F.col("postcode") >= 900) & (F.col("postcode") <= 999))
)

print(tbl_consumer.select("state").distinct())
print(tbl_consumer.select("gender").distinct())


print("Number of unique consumer_id: "+str(tbl_consumer.select("consumer_id").distinct().count()))
print("Number of counts after preprocessing: " + str(tbl_consumer.count()))
tbl_consumer = tbl_consumer.withColumnRenamed("name","consumer_name")

tbl_consumer.printSchema()
tbl_consumer

Number of counts before preprocessing: 499999
+-----+
|state|
+-----+
|   NT|
|  ACT|
|   SA|
|  TAS|
|   WA|
|  QLD|
|  VIC|
|  NSW|
+-----+

+-----------+
|     gender|
+-----------+
|Undisclosed|
|     Female|
|       Male|
+-----------+



Number of unique consumer_id: 499690
Number of counts after preprocessing: 499690
root
 |-- consumer_name: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postcode: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- consumer_id: string (nullable = true)



consumer_name,state,postcode,gender,consumer_id
Yolanda Williams,WA,6935,Female,1195503
Mary Smith,NSW,2782,Female,179208
Jill Jones MD,NT,862,Female,1194530
Lindsay Jimenez,NSW,2780,Female,154128
Rebecca Blanchard,WA,6355,Female,712975
Karen Chapman,NSW,2033,Female,407340
Andrea Jones,QLD,4606,Female,511685
Stephen Williams,WA,6056,Male,448088
Stephanie Reyes,NSW,2482,Female,650435
Jillian Gonzales,VIC,3220,Female,1058499


In [5]:
merchant_fraud = spark.read.option("header", True).csv("../data/tables/project-2-bnpl-tables-part1/tables/merchant_fraud_probability.csv")
print(merchant_fraud.count())

merchant_fraud = merchant_fraud.filter((F.col("fraud_probability")>=0) &(F.col("fraud_probability")<=100))
# Type casting
merchant_fraud = merchant_fraud.withColumn("fraud_probability", merchant_fraud["fraud_probability"].cast(DoubleType()))
merchant_fraud = merchant_fraud.withColumn("order_datetime", merchant_fraud["order_datetime"].cast(DateType()))


#merchant_fraud = merchant_fraud.withColumn("order_datetime", F.date_format(merchant_fraud["order_datetime"], "yyyy-MM"))    # I dropped the date here cause we said we only need month.
merchant_fraud.printSchema()
merchant_fraud = merchant_fraud.withColumnRenamed("fraud_probability", "merchant_fraud_probability")
merchant_fraud.select([F.sum(F.col(i).isNull().cast("int")).alias(i) for i in merchant_fraud.columns]).show()



114
root
 |-- merchant_abn: string (nullable = true)
 |-- order_datetime: date (nullable = true)
 |-- fraud_probability: double (nullable = true)

+------------+--------------+--------------------------+
|merchant_abn|order_datetime|merchant_fraud_probability|
+------------+--------------+--------------------------+
|           0|             0|                         0|
+------------+--------------+--------------------------+



In [6]:
# copied from Alan
tbl_merchants = spark.read.parquet("../data/tables/project-2-bnpl-tables-part1/tables/tbl_merchants.parquet")
tbl_merchants = tbl_merchants.withColumn('tags', F.regexp_replace('tags', r'\(', r'\[')) \
    .withColumn('tags', F.lower(F.regexp_replace('tags', r'\)', r'\]')))

tbl_merchants = tbl_merchants.withColumn('tags1', (F.regexp_extract('tags', r'\[(\[[^\]]*\])[^\[]*\[([^\]]*)\][^\[]*\[take rate: ([^\]]*)\]\]', idx=1)))
tbl_merchants = tbl_merchants.withColumn('tags2', (F.regexp_extract('tags', r'\[(\[[^\]]*\])[^\[]*\[([^\]]*)\][^\[]*\[take rate: ([^\]]*)\]\]', idx=2)))
tbl_merchants = tbl_merchants.withColumn('tags3', (F.regexp_extract('tags', r'\[(\[[^\]]*\])[^\[]*\[([^\]]*)\][^\[]*\[take rate: ([^\]]*)\]\]', idx=3)).cast(DoubleType()))
tbl_merchants.sort('tags3').show(truncate=False)
tbl_merchants = tbl_merchants.withColumnRenamed("name", "merchant_name")


+----------------------------+-----------------------------------------------------------------------------------------------------------------+------------+---------------------------------------------------------------------------------------+-----+-----+
|name                        |tags                                                                                                             |merchant_abn|tags1                                                                                  |tags2|tags3|
+----------------------------+-----------------------------------------------------------------------------------------------------------------+------------+---------------------------------------------------------------------------------------+-----+-----+
|Lacus LLP                   |[[books, periodicals, and newspapers], [e], [take rate: 0.10]]                                                   |55622580330 |[books, periodicals, and newspapers]                                 

For here, I looked up what is considered a legit ABN number, and this is code that will execute it, but when its done you can see that there is only 5 entries left so I'm not sure if we should leave it in or not.

In [7]:
'''
def valid_abn(abn_number):
    abn_number = str(abn_number)
    
    if len(abn_number) != 11 or not abn_number.isdigit():
        return False 
    
    digits = [int(i) for i in abn_number]
    
    digits[0] -= 1
    
    weights = [10, 1, 3, 5, 7, 9, 11, 13, 15, 17, 19]
    
    total = sum(d * w for d, w in zip(digits, weights))
    
    return total % 89 == 0

validate_abn_udf = F.udf(valid_abn, BooleanType())
'''

'\ndef valid_abn(abn_number):\n    abn_number = str(abn_number)\n    \n    if len(abn_number) != 11 or not abn_number.isdigit():\n        return False \n    \n    digits = [int(i) for i in abn_number]\n    \n    digits[0] -= 1\n    \n    weights = [10, 1, 3, 5, 7, 9, 11, 13, 15, 17, 19]\n    \n    total = sum(d * w for d, w in zip(digits, weights))\n    \n    return total % 89 == 0\n\nvalidate_abn_udf = F.udf(valid_abn, BooleanType())\n'

In [8]:
'''
test = merchant_fraud.withColumn("valid_abn", validate_abn_udf(merchant_fraud["merchant_abn"]))
test = test.filter(F.col("valid_abn") == True)
test.show()
'''

'\ntest = merchant_fraud.withColumn("valid_abn", validate_abn_udf(merchant_fraud["merchant_abn"]))\ntest = test.filter(F.col("valid_abn") == True)\ntest.show()\n'

In [9]:
if not os.path.exists('../data/curated/'):
    os.makedirs('../data/curated/')
    
if not os.path.exists('../data/curated/'):
    os.makedirs('../data/curated/')

merchant_fraud.write.mode('overwrite').parquet("../data/curated/merchant_fraud")
tbl_consumer.write.mode('overwrite').parquet("../data/curated/tbl_consumer")
tbl_merchants.write.mode('overwrite').parquet("../data/curated/tbl_merchants")

## Joining to one big table.

In [10]:
tbl_consumer = spark.read.parquet("../data/curated/tbl_consumer")
merchant_fraud = spark.read.parquet("../data/curated/merchant_fraud")
consumer_fraud = spark.read.parquet("../data/curated/consumer_fraud")
consumer_details = spark.read.parquet("../data/curated/consumer_details")
tbl_merchants = spark.read.parquet("../data/curated/tbl_merchants")

In [11]:
full_transactions = transactions.join(consumer_details, on="user_id",how="left") \
                   .join(tbl_consumer, on="consumer_id",how="left") \
                   .join(consumer_fraud, on=["user_id", "order_datetime"],how="left") \
                   .join(tbl_merchants, on="merchant_abn",how="left") \
                   .join(merchant_fraud, on=["order_datetime", "merchant_abn"], how="left")


In [12]:
full_transactions.show()

+--------------+------------+-------+-----------+------------------+--------------------+----------------+-----+--------+------+-----------------+--------------------+--------------------+--------------------+-----+-----+--------------------------+
|order_datetime|merchant_abn|user_id|consumer_id|      dollar_value|            order_id|   consumer_name|state|postcode|gender|fraud_probability|       merchant_name|                tags|               tags1|tags2|tags3|merchant_fraud_probability|
+--------------+------------+-------+-----------+------------------+--------------------+----------------+-----+--------+------+-----------------+--------------------+--------------------+--------------------+-----+-----+--------------------------+
|    2021-08-20| 62191208634|  18478|     651338|63.255848959735246|949a63c8-29f7-4ab...|     James Smith|  TAS|    7001|  Male|             NULL|Cursus Non Egesta...|[[furniture, home...|[furniture, home ...|    c| 2.17|                      NULL|
|   

In [15]:
full_transactions.filter(F.col('merchant_name').isNull()).show()

+--------------+------------+-------+-----------+------------------+--------------------+------------------+-----+--------+-----------+-----------------+-------------+----+-----+-----+-----+--------------------------+
|order_datetime|merchant_abn|user_id|consumer_id|      dollar_value|            order_id|     consumer_name|state|postcode|     gender|fraud_probability|merchant_name|tags|tags1|tags2|tags3|merchant_fraud_probability|
+--------------+------------+-------+-----------+------------------+--------------------+------------------+-----+--------+-----------+-----------------+-------------+----+-----+-----+-----+--------------------------+
|    2021-08-20| 29566626791|      8|     448088| 74.15732460440282|71a81652-cc91-4bf...|  Stephen Williams|   WA|    6056|       Male|             NULL|         NULL|NULL| NULL| NULL| NULL|                      NULL|
|    2021-08-20| 32234779638|  18490|    1158069|107.14809429376949|20149572-a55b-41f...|       Sara Holmes|   WA|    6425|     

In [16]:
full_transactions.write.mode('overwrite').parquet("../data/curated/full_transcation")

24/09/04 23:46:32 ERROR Executor: Exception in task 11.0 in stage 68.0 (TID 764)
java.lang.IllegalArgumentException: java.net.URISyntaxException: Relative path in absolute URI: .part-00001-cfff5211-ef46-4b99-a6ed-63c9b2c7fe15.c000.snappy.parquet:Zone.Identifier.crc
	at org.apache.hadoop.fs.Path.initialize(Path.java:263)
	at org.apache.hadoop.fs.Path.<init>(Path.java:221)
	at org.apache.hadoop.fs.Path.<init>(Path.java:129)
	at org.apache.hadoop.fs.ChecksumFileSystem.getChecksumFile(ChecksumFileSystem.java:107)
	at org.apache.hadoop.fs.ChecksumFileSystem$ChecksumFSInputChecker.<init>(ChecksumFileSystem.java:163)
	at org.apache.hadoop.fs.ChecksumFileSystem.open(ChecksumFileSystem.java:372)
	at org.apache.hadoop.fs.FileSystem.open(FileSystem.java:976)
	at org.apache.parquet.hadoop.util.HadoopInputFile.newStream(HadoopInputFile.java:69)
	at org.apache.parquet.hadoop.ParquetFileReader.<init>(ParquetFileReader.java:796)
	at org.apache.parquet.hadoop.ParquetFileReader.open(ParquetFileReader.ja

Py4JJavaError: An error occurred while calling o387.parquet.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 8 in stage 68.0 failed 1 times, most recent failure: Lost task 8.0 in stage 68.0 (TID 761) (172.24.73.29 executor driver): java.lang.IllegalArgumentException: java.net.URISyntaxException: Relative path in absolute URI: .part-00001-cfff5211-ef46-4b99-a6ed-63c9b2c7fe15.c000.snappy.parquet:Zone.Identifier.crc
	at org.apache.hadoop.fs.Path.initialize(Path.java:263)
	at org.apache.hadoop.fs.Path.<init>(Path.java:221)
	at org.apache.hadoop.fs.Path.<init>(Path.java:129)
	at org.apache.hadoop.fs.ChecksumFileSystem.getChecksumFile(ChecksumFileSystem.java:107)
	at org.apache.hadoop.fs.ChecksumFileSystem$ChecksumFSInputChecker.<init>(ChecksumFileSystem.java:163)
	at org.apache.hadoop.fs.ChecksumFileSystem.open(ChecksumFileSystem.java:372)
	at org.apache.hadoop.fs.FileSystem.open(FileSystem.java:976)
	at org.apache.parquet.hadoop.util.HadoopInputFile.newStream(Hado